In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Problem Statement**
In mnist dataset contains thousands of handwritten digit images. Our goal is predicat the handwritten image to digit.


In [ ]:
## Loading the necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import losses
from keras import optimizers
from keras.layers import Dropout, MaxPooling2D, Dense, BatchNormalization, Flatten, Conv2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
## Reading the train dataset
df= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
df.head()

In [ ]:
# checking the shape of the data
print('Shape of the dataset {}'.format(df.shape))

In [ ]:
# Checking the unique label
print('Unique labels of the dataset: {}'.format(df['label'].unique()))
print('Number of unique classes: {}'.format(len(df['label'].unique())))

In [ ]:
# seperating the dependent and independent features
x = df.drop('label', axis=1)
y = df['label']
x

In [ ]:
# display the label
y

In [ ]:
# spliting the data into test and train
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state=0)


In [ ]:
print('Shape of the x_train set: {}'.format(x_train.shape))
print('Shape of the y_train set: {}'.format(y_train.shape))
print('Shape of the x_test set: {}'.format(x_val.shape))
print('Shape of the y_test set: {}'.format(y_val.shape))

In [ ]:
# reshaping the data
x_train = x_train.values.reshape(-1,28,28,1)
x_val = x_val.values.reshape(-1,28,28,1)
print('Shape of the x_trian image: {}'.format(x_train.shape[1:]))
print('Shape of the x_test image: {}'.format(x_val.shape[1:]))

In [ ]:
# Displaying the 10 handwritten digit
for i in range(5):
    plt.imshow(x_train[i])
    plt.show()

In [ ]:
# normalize the pixel
x_train = x_train.astype('float32')/255.0
x_val = x_val.astype('float32')/255.0

In [ ]:
def model_creation(shape_input):
    model = Sequential([
        
        Dense(300,activation='relu',input_shape = shape_input),
        Dropout(0.20),
        Dense(100,activation='relu'),
        Dropout(0.20),
        Dense(64,activation='relu'),
        Dropout(0.20),
        Flatten(),
        Dense(256,activation='relu'),
        Dropout(0.20),
        Dense(10,activation='softmax')
    ])
    sgd = optimizers.Adam(lr=1e-3)
    model.compile(optimizer = sgd, loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
    return model

In [ ]:
# build a model
shape_input = x_train.shape[1:]
model = model_creation(shape_input)

In [ ]:
# print the model summary
model.summary()

In [ ]:
# fti the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=50, verbose=2)

## **Model Evaluation**

In [ ]:
## plot function
def plot_model_evaluation(history,materics,title):
    plt.plot(history.history[materics])
    plt.plot(history.history['val_'+materics])
    plt.title(title)
    plt.legend(['train','val'])
    plt.xlabel('Epochs')
    plt.ylabel(materics)
    plt.show()

In [ ]:
# ploting the accuracy
plot_model_evaluation(history,'accuracy', 'Model Accuracy')

In [ ]:
# ploting the model loss
plot_model_evaluation(history, 'loss', 'Model Loss')

In [ ]:
x_test = test.copy()
x_test = x_test.values.reshape(-1,28,28,1)
x_test = x_test.astype('float32')/255.0

In [ ]:
 y_pred = model.predict(x_test)

In [ ]:
my_submition = pd.DataFrame({'ImageId':test.index.values + 1, 'Label':np.argmax(y_pred, axis=1)})

In [ ]:
my_submition.to_csv('digit.csv', index=False)